# Тестовое задание по алгоритмам.
Дана таблица с показаниями счетчиков электроэнергии за 2 квартал (апрель, май, июнь) в файле "Показания.csv".  
В таблице строка, содержащая номер квартиры - относится к квартирному счетчику. Строка, где № квартиры равен 0 - к общедомовому.  

Необходимо:
1. Рассчитать потребление электроэнергии за период апрель-май и май-июнь для всех счетчиков (как разность соответствующих показаний). Сформировать новый файл "Потребление.csv", записать в него и исходные, и рассчитанные данные.
2. Определить, в каком доме, возможно, переданы неверные показания. Критерий: сумма потребления по общедомовому счетчику за оба периода не равна сумме потребления за оба периода всех квартирных счетчиков этого дома.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('indications.csv', sep=';')
df.head(5)

,№ строки,Улица,№ дома,№ Квартиры,Апрель,Май,Июнь
0,1,Пушкина,1,0,22237,22637,23035
1,2,Пушкина,1,1,7409,7559,7702
2,3,Пушкина,1,2,7411,7541,7664
3,4,Пушкина,1,3,7417,7537,7669
4,5,Пушкина,2,0,14811,15086,15407


In [3]:
df['Адрес'] = df['Улица'] + ' ' + df['№ дома'].astype(str)
df = df[['№ строки', 'Адрес', '№ Квартиры', 'Апрель', 'Май', 'Июнь']]
df.head(5)

,№ строки,Адрес,№ Квартиры,Апрель,Май,Июнь
0,1,Пушкина 1,0,22237,22637,23035
1,2,Пушкина 1,1,7409,7559,7702
2,3,Пушкина 1,2,7411,7541,7664
3,4,Пушкина 1,3,7417,7537,7669
4,5,Пушкина 2,0,14811,15086,15407


Рассчитаем потребление электроэнергии за период апрель-май и май-июнь для всех счетчиков (как разность соответствующих показаний). Сформируем новый файл "Потребление.csv", зашем в него и исходные, и рассчитанные данные.

In [4]:
cons_df = df.copy()
indic_cols = ['Апрель', 'Май', 'Июнь']
cons_cols = []

for i in range(1, len(indic_cols)):
    cons_df[f'Потребление {indic_cols[i-1]}-{indic_cols[i]}'] = df[indic_cols[i]] - df[indic_cols[i-1]]

cons_df.head(5)

,№ строки,Адрес,№ Квартиры,Апрель,Май,Июнь,Потребление Апрель-Май,Потребление Май-Июнь
0,1,Пушкина 1,0,22237,22637,23035,400,398
1,2,Пушкина 1,1,7409,7559,7702,150,143
2,3,Пушкина 1,2,7411,7541,7664,130,123
3,4,Пушкина 1,3,7417,7537,7669,120,132
4,5,Пушкина 2,0,14811,15086,15407,275,321


In [5]:
cons_df.to_csv('consumptions.csv', sep=';')

Определяем, в каком доме, возможно, переданы неверные показания. Критерий: сумма потребления по общедомовому счетчику за оба периода не равна сумме потребления за оба периода всех квартирных счетчиков этого дома.

In [6]:
house_cons_df = cons_df[cons_df['№ Квартиры'] == 0].reset_index().iloc[:, 2:]
apartments_cons_df = cons_df[cons_df['№ Квартиры'] != 0].iloc[:, 1:]

In [7]:
sum_apartments_cons_df = apartments_cons_df.groupby(by='Адрес', sort=False) \
                                           .agg({'№ Квартиры': 'count',
                                                 'Апрель': 'sum',
                                                 'Май': 'sum',
                                                 'Июнь': 'sum',
                                                 'Потребление Апрель-Май': 'sum',
                                                 'Потребление Май-Июнь': 'sum',}) \
                                           .rename(columns={'№ Квартиры': 'Количество квартир'}) \
                                           .reset_index()

In [8]:
house_cons_df

,Адрес,№ Квартиры,Апрель,Май,Июнь,Потребление Апрель-Май,Потребление Май-Июнь
0,Пушкина 1,0,22237,22637,23035,400,398
1,Пушкина 2,0,14811,15086,15407,275,321
2,Пушкина 3,0,22231,22629,23017,398,388
3,Пушкина 4,0,22233,22666,23147,433,481
4,Некрасова 1,0,14818,15139,15405,321,266
5,Некрасова 2,0,22237,22690,23111,453,421
6,Некрасова 3,0,14811,15077,15352,266,275
7,Лермонтова 1,0,14822,15056,15322,234,266
8,Лермонтова 2,0,22240,22624,23024,384,400
9,Лермонтова 3,0,14815,15092,15293,277,201


In [9]:
sum_apartments_cons_df

,Адрес,Количество квартир,Апрель,Май,Июнь,Потребление Апрель-Май,Потребление Май-Июнь
0,Пушкина 1,3,22237,22637,23035,400,398
1,Пушкина 2,2,14811,15086,15407,275,321
2,Пушкина 3,3,22231,22629,23017,398,388
3,Пушкина 4,3,22233,22621,23042,388,421
4,Некрасова 1,2,14818,15139,15405,321,266
5,Некрасова 2,3,22237,22658,23056,421,398
6,Некрасова 3,2,14811,15077,15352,266,275
7,Лермонтова 1,2,14822,15023,15289,201,266
8,Лермонтова 2,3,22240,22624,23024,384,400
9,Лермонтова 3,2,14815,15092,15293,277,201


In [10]:
cond_1 = house_cons_df['Потребление Апрель-Май'] != sum_apartments_cons_df['Потребление Апрель-Май']
cond_2 = house_cons_df['Потребление Май-Июнь'] != sum_apartments_cons_df['Потребление Май-Июнь']

df_1 = house_cons_df.loc[cond_1 | cond_2, ['Адрес', 'Потребление Апрель-Май', 'Потребление Май-Июнь']]
df_1['Способ вычисления'] = 'общедомовой счетчик'
df_2 = sum_apartments_cons_df.loc[cond_1 | cond_2, ['Адрес', 'Потребление Апрель-Май', 'Потребление Май-Июнь']]
df_2['Способ вычисления'] = 'сумма квартирных счетчиков'

compare_df = pd.concat([df_1, df_2]).sort_values(['Адрес', 'Способ вычисления']).set_index(['Адрес', 'Способ вычисления'])
compare_df

Потребление Апрель-Май  \
Адрес        Способ вычисления                                    
Лермонтова 1 общедомовой счетчик                            234   
             сумма квартирных счетчиков                     201   
Некрасова 2  общедомовой счетчик                            453   
             сумма квартирных счетчиков                     421   
Пушкина 4    общедомовой счетчик                            433   
             сумма квартирных счетчиков                     388   

                                         Потребление Май-Июнь  
Адрес        Способ вычисления                                 
Лермонтова 1 общедомовой счетчик                          266  
             сумма квартирных счетчиков                   266  
Некрасова 2  общедомовой счетчик                          421  
             сумма квартирных счетчиков                   398  
Пушкина 4    общедомовой счетчик                          481  
             сумма квартирных счетчиков                   421

Как видим из сравнительной таблицы, расхождения в показаниях потребеления с общедомовым счетчиком наблюдались в домах: Пушкина 4, Некрасова 2, Лермонтова 1. В последнем из них расхождения были только в первом периоде.